
<br>
================================<br>
Segment human cells (in mitosis)<br>
================================<br>
In this example, we analyze a microscopy image of human cells. We use data<br>
provided by Jason Moffat [1]_ through<br>
`CellProfiler <https://cellprofiler.org/examples/#human-cells>`_.<br>
.. [1] Moffat J, Grueneberg DA, Yang X, Kim SY, Kloepfer AM, Hinkle G, Piqani<br>
       B, Eisenhaure TM, Luo B, Grenier JK, Carpenter AE, Foo SY, Stewart SA,<br>
       Stockwell BR, Hacohen N, Hahn WC, Lander ES, Sabatini DM, Root DE<br>
       (2006) "A lentiviral RNAi library for human and mouse genes applied to<br>
       an arrayed viral high-content screen" Cell, 124(6):1283-98.<br>
       PMID: 16564017<br>
       :DOI:`10.1016/j.cell.2006.01.040`<br>


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import ndimage as ndi

In [ ]:
from skimage import (
    color, feature, filters, measure, morphology, segmentation, util
)
from skimage.data import human_mitosis

In [ ]:
image = human_mitosis()

In [ ]:
fig, ax = plt.subplots()
ax.imshow(image, cmap='gray')
ax.set_title('Microscopy image of human cells stained for nuclear DNA')
plt.show()

###################################################################<br>
We can see many cell nuclei on a dark background. Most of them are smooth<br>
and have an elliptical shape. However, we can distinguish some brighter<br>
spots corresponding to nuclei undergoing<br>
`mitosis <https://en.wikipedia.org/wiki/Mitosis>`_ (cell division).

###################################################################<br>
Another way of visualizing a grayscale image is contour plotting:

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
qcs = ax.contour(image, origin='image')
ax.set_title('Contour plot of the same raw image')
# sphinx_gallery_thumbnail_number = 2
plt.show()

###################################################################<br>
The contour lines are drawn at these levels:

In [ ]:
qcs.levels

###################################################################<br>
Each level has, respectively, the following number of segments:

In [ ]:
[len(seg) for seg in qcs.allsegs]

###################################################################<br>
Estimate the mitotic index<br>
==========================<br>
Cell biology uses the<br>
`mitotic index <https://en.wikipedia.org/wiki/Mitotic_index>`_<br>
to quantify cell division and,<br>
hence, cell proliferation. By definition, it is the ratio of cells in<br>
mitosis over the total number of cells. To analyze the above image,<br>
we are thus interested in two thresholds: one separating the nuclei from the<br>
background, the other separating the dividing nuclei (brighter spots) from<br>
the non-dividing nuclei.<br>
To separate these three different classes of pixels, we<br>
resort to :ref:`sphx_glr_auto_examples_segmentation_plot_multiotsu.py`.

In [ ]:
thresholds = filters.threshold_multiotsu(image, classes=3)
regions = np.digitize(image, bins=thresholds)

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(10, 5))
ax[0].imshow(image)
ax[0].set_title('Original')
ax[0].axis('off')
ax[1].imshow(regions)
ax[1].set_title('Multi-Otsu thresholding')
ax[1].axis('off')
plt.show()

###################################################################<br>
Since there are overlapping nuclei, thresholding is not enough to segment<br>
all the nuclei.<br>
If it were, we could readily compute a mitotic index for this<br>
sample:

In [ ]:
cells = image > thresholds[0]
dividing = image > thresholds[1]
labeled_cells = measure.label(cells)
labeled_dividing = measure.label(dividing)
naive_mi = labeled_dividing.max() / labeled_cells.max()
print(naive_mi)

###################################################################<br>
Whoa, this can't be! The number of dividing nuclei

In [ ]:
print(labeled_dividing.max())

###################################################################<br>
is overestimated, while the total number of cells

In [ ]:
print(labeled_cells.max())

###################################################################<br>
is underestimated.

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(15, 5))
ax[0].imshow(image)
ax[0].set_title('Original')
ax[0].axis('off')
ax[2].imshow(cells)
ax[2].set_title('All nuclei?')
ax[2].axis('off')
ax[1].imshow(dividing)
ax[1].set_title('Dividing nuclei?')
ax[1].axis('off')
plt.show()

###################################################################<br>
Count dividing nuclei<br>
=====================<br>
Clearly, not all connected regions in the middle plot are dividing nuclei.<br>
On one hand, the second threshold (value of ``thresholds[1]``) appears to be<br>
too low to separate those very bright areas corresponding to dividing nuclei<br>
from relatively bright pixels otherwise present in many nuclei. On the other<br>
hand, we want a smoother image, removing small spurious objects and,<br>
possibly, merging clusters of neighbouring objects (some could correspond to<br>
two nuclei emerging from one cell division). In a way, the segmentation<br>
challenge we are facing with dividing nuclei is the opposite of that with<br>
(touching) cells.

###################################################################<br>
To find suitable values for thresholds and filtering parameters, we proceed<br>
by dichotomy, visually and manually.

In [ ]:
higher_threshold = 125
dividing = image > higher_threshold

In [ ]:
smoother_dividing = filters.rank.mean(util.img_as_ubyte(dividing),
                                      morphology.disk(4))

In [ ]:
binary_smoother_dividing = smoother_dividing > 20

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
ax.imshow(binary_smoother_dividing)
ax.set_title('Dividing nuclei')
ax.axis('off')
plt.show()

###################################################################<br>
We are left with

In [ ]:
cleaned_dividing = measure.label(binary_smoother_dividing)
print(cleaned_dividing.max())

###################################################################<br>
dividing nuclei in this sample.

###################################################################<br>
Segment nuclei<br>
==============<br>
To separate overlapping nuclei, we resort to<br>
:ref:`sphx_glr_auto_examples_segmentation_plot_watershed.py`.<br>
To visualize the segmentation conveniently, we colour-code the labelled<br>
regions using the `color.label2rgb` function, specifying the background<br>
label with argument `bg_label=0`.

In [ ]:
distance = ndi.distance_transform_edt(cells)

In [ ]:
local_max_coords = feature.peak_local_max(distance, min_distance=7)
local_max_mask = np.zeros(distance.shape, dtype=bool)
local_max_mask[tuple(local_max_coords.T)] = True
markers = measure.label(local_max_mask)

In [ ]:
segmented_cells = segmentation.watershed(-distance, markers, mask=cells)

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(10, 5))
ax[0].imshow(cells, cmap='gray')
ax[0].set_title('Overlapping nuclei')
ax[0].axis('off')
ax[1].imshow(color.label2rgb(segmented_cells, bg_label=0))
ax[1].set_title('Segmented nuclei')
ax[1].axis('off')
plt.show()

###################################################################<br>
Additionally, we may use function `color.label2rgb` to overlay the original<br>
image with the segmentation result, using transparency (alpha parameter).

In [ ]:
color_labels = color.label2rgb(segmented_cells, image, alpha=0.4, bg_label=0)

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
ax.imshow(color_labels)
ax.set_title('Segmentation result over raw image')
plt.show()

###################################################################<br>
Finally, we find a total number of

In [ ]:
print(segmented_cells.max())

###################################################################<br>
cells in this sample. Therefore, we estimate the mitotic index to be:

In [ ]:
print(cleaned_dividing.max() / segmented_cells.max())